<a href="https://colab.research.google.com/github/DhruvDave12/CS362-LAB1/blob/main/Lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [111]:
import numpy as np
from copy import deepcopy
import random
import time
import os, psutil
import resource

In [112]:
def manhattanHeuristic(curr_state, goal_state):

    goal_indices = np.argsort(goal_state.reshape(-1,1), axis=0)

    curr_indices = np.argsort(curr_state.reshape(-1,1), axis=0)

    # Finding the x and y distance from the goal indice to the curr state indice
    x = (abs(goal_indices // 3 - curr_indices // 3))
    y = (abs(goal_indices % 3 - curr_indices % 3))

    mDist = np.sum(x + y)
    # returning the manhattan distance
    return mDist

In [113]:
def misplacedTilesHeuristic(curr_state, goal_state):
  
    # h is the number of misplaced tiles between the curr state and goal state
    misplacedTiles = np.sum(curr_state != goal_state)
    return misplacedTiles

In [114]:
# This routine generates a random instance of the 8-puzzle at the given depth (d) from the goal state.
def generate_instance(goal_state, depth):

    x_prev, y_prev = np.array(np.where(goal_state == 0)).reshape(-1)
    curr_state = np.copy(goal_state)

    visited = np.array([curr_state.reshape(-1)])
    n = depth
 
    while(n):
        # Here we are storing the possible positions according to the conditions so that we are constrained inside the puzzle
        possible_states = []
        if x_prev > 0:
            possible_states.append([x_prev-1, y_prev])
        if x_prev < 2:
            possible_states.append([x_prev+1, y_prev])
        if y_prev > 0:
            possible_states.append([x_prev, y_prev-1])
        if y_prev < 2:
            possible_states.append([x_prev, y_prev+1])

        # choosing a random state from the possible states
        x_new, y_new = random.choice(possible_states)

        curr_state[x_new, y_new], curr_state[x_prev, y_prev] = curr_state[x_prev, y_prev], curr_state[x_new, y_new]

        if curr_state.reshape(-1).tolist() not in visited.tolist():
            visited = np.vstack((visited, curr_state.reshape(-1)))
            n -= 1
            x_prev, y_prev = x_new, y_new
        else:
            # go back to the prev state and do the same
            curr_state[x_new, y_new], curr_state[x_prev, y_prev] = curr_state[x_prev, y_prev], curr_state[x_new, y_new]
    return curr_state

In [115]:
GOAL_STATE = np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8]])
print("Goal State:\n", GOAL_STATE)

Goal State:
 [[0 1 2]
 [3 4 5]
 [6 7 8]]


In [116]:
CURR_STATE_TEST = generate_instance(GOAL_STATE,10)

In [117]:
# Heuristic 2
print("Value from Manhattan Tiles Heuristic:", manhattanHeuristic(CURR_STATE_TEST, GOAL_STATE))

Value from Manhattan Tiles Heuristic: 8


In [118]:
# Heuristic 2
print("Value from Misplaced Tiles Heuristic:", misplacedTilesHeuristic(CURR_STATE_TEST, GOAL_STATE))

Value from Misplaced Tiles Heuristic: 5


In [119]:
# A generic routine to simply find the position of an element in the matrix
def find_pos(matrix,value):
    if value < 0 or value > 8:
      raise Exception ("Give the value is out of range")
    else:
      # print(matrix, value)
      return np.array(np.where(matrix == value )).reshape(-1)

In [120]:
# Playing the move of swapping the tile of the 8-puzzle game according to the desired state that our graph search agent gave us
def swap_tiles(matrix,next_pos):
    row,col=find_pos(matrix,0)
    matrix[row, col], matrix[next_pos[0], next_pos[1]] = matrix[next_pos[0], next_pos[1]], matrix[row, col]

    return matrix

In [121]:
# This gives the possible moves available from the current state that makes sure we are under our constraints.
def possible_moves(current_state):
    row,col=find_pos(current_state,0)
    ans_list=[]
    if(row>0):
        ans_list.append([row-1,col])
    if(row<2):
        ans_list.append([row+1,col])
    if(col>0):
        ans_list.append([row,col-1])
    if(col<2):
        ans_list.append([row,col+1])
    return ans_list

In [122]:
#  visited array to track the visited states
visited = np.array([0]*9)

In [151]:
# routine that uses all the above routines and solve the 8-puzzle problem
def solver(curr_state,visited,technique):
    next_possible=[]
    heurestic_val=[]
    choosen_matrix=curr_state
    visited = np.vstack((visited, choosen_matrix.reshape(-1)))
    if (choosen_matrix == GOAL_STATE).all():
        # print("GOAL STATE REACHED")
        return 
    for i in possible_moves(curr_state):
       if np.array(i).reshape(-1).tolist() not in visited.tolist():
        matrix2=deepcopy(curr_state)
        temp=swap_tiles(matrix2,i)
        next_possible.append(temp)

        
        c = 0
        if(technique == "manhattan"):
          c = manhattanHeuristic(temp,GOAL_STATE)
        else: 
          c = misplacedTilesHeuristic(temp,GOAL_STATE)
        # storing the heuristic vals here for all the possible moves from the current state
        heurestic_val.append(c)
    
    # finding the move that had the minimum heuristic
    for k in range(len(next_possible)):
        if(heurestic_val[k]==min(heurestic_val)):
            visited = np.vstack((visited, next_possible[k].reshape(-1)))
            choosen_matrix=next_possible[k]

    if (choosen_matrix == GOAL_STATE).all():
        # print("GOAL STATE REACHED")
        return
    else:
        solver(choosen_matrix,visited,technique)
        

In [181]:
# Here we can change the second parameter that will give us the instance for a different depth we will analyze it.
DEPTH = 4
CURR_STATE = generate_instance(GOAL_STATE, DEPTH)

In [182]:
print("Current State at desired depth is: ",CURR_STATE)

Current State at desired depth is:  [[1 4 2]
 [6 3 5]
 [0 7 8]]


In [183]:
%%timeit
solver(CURR_STATE, visited, "manhattan")

1.05 ms ± 208 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [184]:
visited2 = np.array([0]*9)

In [185]:
%%timeit
solver(CURR_STATE, visited2, "misplaced")

805 µs ± 165 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
